In [2]:
#data at https://moneypuck.com/data.htm
import pandas as pd
import os
import json

In [6]:
folder_path = '/Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data'
csv_folder = os.path.join(folder_path, 'csvs')
output_folder = os.path.join(folder_path, 'jsons')

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Loop through all CSV files in the directory
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        df = pd.read_csv(file_path)

        # Convert to list-of-dict format
        data_as_json = df.to_dict(orient='records')

        # Define output filename (.txt with same name as .csv)
        base_name = os.path.splitext(filename)[0].replace(' ', '_')
        output_file = os.path.join(output_folder, f"{base_name}.txt")

        # Save JSON to .txt
        with open(output_file, 'w') as f:
            json.dump(data_as_json, f, indent=4)

        print(f"Saved {output_file}")

Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies.txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies_(4).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(15).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/skaters_(5).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/teams_(2).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/goalies_(14).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(13).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/data/jsons/lines_(9).txt
Saved /Users/carlg/Desktop/Code/School/Columbia/DL/nhl_rag_chatbot/da

In [3]:
# import json
# # df = df.drop(columns=['team.1'], errors='ignore')

# # # Convert each row into a readable paragraph
# # def row_to_text(row):
# #     parts = [f"{col}: {row[col]}" for col in df.columns]
# #     return ", ".join(parts)

# # # Combine all rows into one big string
# # full_text = '\n\n'.join(df.apply(row_to_text, axis=1))

# # # Save to text file
# # with open('nhl_stats_full.txt', 'w') as f:
# #     f.write(full_text)
    
# # df = df.drop(columns=['team.1'], errors='ignore')

# # Convert DataFrame to list of dicts (each row is a dict)
# data_as_json = df.to_dict(orient='records')

# # Save to text file in JSON format
# with open('nhl_stats_full.txt', 'w') as f:
#     json.dump(data_as_json, f, indent=4)